In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.messages import HumanMessage

from podflix.graph.mock import compiled_graph

inputs = {"messages": [HumanMessage(content="Hello")]}

In [ ]:
async for msg, metadata in compiled_graph.astream(
    {"messages": [HumanMessage(content="Hello")]}, stream_mode="messages"
):
    if (
        msg.content
        and not isinstance(msg, HumanMessage)
        and metadata["langgraph_node"] == "mock_answer"
    ):
        print(msg.content)

In [ ]:
from pprint import pprint  # noqa: F401

from langchain.schema.runnable.config import RunnableConfig

graph_runnable_config = RunnableConfig(
    callbacks=[],
    recursion_limit=10,
)

streamable_node_names = [
    "mock_answer",
]


async for event in compiled_graph.astream_events(
    inputs,
    config=graph_runnable_config,
    version="v2",
):
    event_kind = event["event"]
    langgraph_node = event["metadata"].get("langgraph_node", None)

    # pprint(event)

    if event_kind == "on_chat_model_stream":
        if langgraph_node not in streamable_node_names:
            continue

        ai_message_content = event["data"]["chunk"].content

        if ai_message_content:
            print(ai_message_content)